In [1]:
#include <algorithm>
#include <numeric>
#include <bitset>
#include <iostream>
#include <sstream>
#include <utility>
#include <vector>

#include <xcpp/xdisplay.hpp>
#include <eigen3/Eigen/Dense>

using Eigen::MatrixXd;

## Matrix shapes

The following are the sizes of the matrices in the equation $Y = (S^T S)^{-1} S^T M$:

Let $\Omega = (S^T S)^{-1} S^T, ~~~\therefore Y = \Omega M$

$
\begin{array}{}
S&: p \times n \\
S^T&: n \times p \\
Y&: n \times 1 \\
M&: n \times 1 \\
\left(S^T S\right)&: n \times n \\
\left(S^T S\right)^{-1}&: n \times n \\
\left(S^T S\right)^{-1} S^T&: n \times p \\
\end{array}
$

In [2]:
{
auto const N = 50;
    
typedef double FLOAT;
typedef Eigen::MatrixXd Matrix;
    
Eigen::MatrixXi S = Eigen::MatrixXi::Ones(N, N);

for (auto i = 0; i < S.cols(); i++) {
    S(i, i) = 0;
}
    
Matrix Y(S.cols(), 1);
    
for (auto i = 0; i < Y.rows(); i++) {
    Y(i) = i + 1;
}

Matrix M(Y.rows(), 1);
M = S.cast<FLOAT>() * Y;

auto S_T = S.transpose();
auto start = std::chrono::high_resolution_clock::now();
auto S_TS_INV = (S_T * S).cast<FLOAT>().inverse();
auto end = std::chrono::high_resolution_clock::now();
auto dur = std::chrono::duration_cast<std::chrono::duration<double, std::ratio<1> > >(end - start);
auto Y_result = S_TS_INV * S_T.cast<FLOAT>() * M;
// std::cout << S << std::endl;
std::cout << Y_result.block<5, 1>(0, 0) << std::endl;
std::cout << "..." << std::endl;
std::cout << Y_result.block<5, 1>(Y_result.rows() - 5, 0) << std::endl;
std::cout << "Matrix inverse calculation took: " << dur.count() << " s" << std::endl;
}

1
2
3
4
5
...
46
47
48
49
50
Matrix inverse calculation took: 4.9e-06 s


# Actuation levels for _sensitive_ channels

For each _sensitive_ electrode, actuation duty cycle can be in the range
$\left[\frac{1}{p}, \frac{p - 1}{p}, \right]$, where $p$ is the number of rows in the switching matrix, $n$ is the number of _sensitive_ channels, and $p \ge n$.

Increasing $p$ can:

 - Reduce the minimum actuation time for sensitive channels
 - Increase the actuation level resolution

## _Sensitive_ channel $S$ column calculation

Let $c_j$ denote the $j^{th}$ column in $S$.

Let $d_j \in [0, 1]$ denote the target duty cycle for the $j^{th}$ sensitive channel.

Let $f(d, p) = \max\left(1, \min\left(dp, p-1\right)\right)$, and $a_j = f(d_j, p)$.

$
\begin{equation*}
c_{i,j} =
\begin{cases}
  1 &\text{if $i = j$ and $a_j \lt j$,} \\
  0 &\text{if $i = j$ and $a_j \ge j$,} \\
  1 &\text{if $j \le a_j = 1$ and $j \le a_j$,} \\
  0 &\text{if $i \ge a_j - 1$,} \\
  1 &\text{otherwise}
\end{cases}
\end{equation*}
$

In [3]:
#include <SwitchingMatrix.h>
#include <SwitchingMatrixEigen.h>

In [4]:
template <typename Matrix>
void test_S(Matrix S) {
    Eigen::MatrixXd Y(S.cols(), 1);

    for (auto i = 0; i < Y.rows(); i++) {
        Y(i) = i + 1;
    }

    Eigen::MatrixXd M(Y.rows(), 1);
    M = S.template cast<double>() * Y;

    auto S_T = S.transpose();
    auto start = std::chrono::high_resolution_clock::now();
    auto S_TS_INV = (S_T * S).template cast<double>().inverse();
    auto end = std::chrono::high_resolution_clock::now();
    auto dur = std::chrono::duration_cast<std::chrono::duration<double, std::ratio<1> > >(end - start);
    auto Y_result = S_TS_INV * S_T.template cast<double>() * M;
    // std::cout << S << std::endl;
    if (Y_result.size() > 10) {
        std::cout << Y_result.template block<5, 1>(0, 0) << std::endl;
        std::cout << "..." << std::endl;
        std::cout << Y_result.template block<5, 1>(Y_result.rows() - 5, 0) << std::endl;
    } else {
        std::cout << Y_result << std::endl;
    }
    std::cout << "Matrix inverse calculation took: " << dur.count() << " s" << std::endl;
}

In [5]:
{
// std::vector<float> levels = {.3, .4, 1., .5, .8, .6, .05};
std::vector<float> levels(10);
std::generate(levels.begin(), levels.end(), [i = 0] () mutable { return 0.1 * (i++ + 1); });
auto p = 2 * levels.size();
    
auto S = dropbot::switching_matrix::switching_matrix(levels, static_cast<int>(p));
// See also: https://eigen.tuxfamily.org/dox/TopicLazyEvaluation.html
auto Omega = ((S.transpose().cast<float>().lazyProduct(S.cast<float>())).inverse() * S.transpose().cast<float>());
    
std::cout << S << std::endl;
std::cout << Omega << std::endl;
    
test_S(S);
}

0 1 1 1 1 1 1 1 1 1
1 0 1 1 1 1 1 1 1 1
1 1 0 1 1 1 1 1 1 1
0 1 1 0 1 1 1 1 1 1
0 1 1 1 0 1 1 1 1 1
0 0 1 1 1 0 1 1 1 1
0 0 1 1 1 1 0 1 1 1
0 0 0 1 1 1 1 0 1 1
0 0 0 1 1 1 1 1 0 1
0 0 0 0 1 1 1 1 1 0
0 0 0 0 1 1 1 1 1 1
0 0 0 0 0 1 1 1 1 1
0 0 0 0 0 1 1 1 1 1
0 0 0 0 0 0 1 1 1 1
0 0 0 0 0 0 1 1 1 1
0 0 0 0 0 0 0 1 1 1
0 0 0 0 0 0 0 1 1 1
0 0 0 0 0 0 0 0 1 1
0 0 0 0 0 0 0 0 1 1
0 0 0 0 0 0 0 0 0 1
  -0.211168    0.626347    0.373653 -0.00689774   -0.155587   -0.121959   -0.130736   -0.199472   -0.181079   -0.109923  -0.0387669   0.0168142   0.0168142 -0.00438856 -0.00438856  -0.0343684  -0.0343684  0.00919686  0.00919686   0.0711557
   0.251974    -0.35276     0.35276    0.240478    0.154787   -0.124878   -0.169602  -0.0688366  -0.0434452  -0.0430454  -0.0426455   -0.139832   -0.139832  -0.0223624  -0.0223624   0.0503829   0.0503829   0.0126957   0.0126957 0.000399896
  0.0113762    0.364856   -0.364856    0.261591   0.0918888    0.161465    0.108823   -0.171322   -0.202231   -0.123978 